In [7]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "20"  # ← 改成你的物理核数
os.environ["OMP_NUM_THREADS"] = "1"


import numpy as np
from sklearn.cluster import KMeans
from utils import *

# 参数设置 Parameter Set
os.environ["OMP_NUM_THREADS"] = "1"
data_Folder = "Simulated_data\\"
K           = 10          # 簇数 Num of cluster
# num_users   = 150 # 用户数 Num of users

train_ratio = 0.72
val_ratio   = 0.08
max_epoch   = 20
save_path   = "Process_1\\CLC\\"
os.makedirs(save_path,exist_ok=True)

In [8]:
for data_idx in range(1,11):
    num =0

    load_data   = np.load(data_Folder+"simulated_load_"+str(data_idx)+".npy") # 加载原始数据 users' load data
    temp_data   = np.load(data_Folder+"beijing_temp_halfhour_99.npy") #加载温度数据 temperaure data
    time_index  = np.arange(1,load_data.shape[1]+1,dtype=load_data.dtype) # 加载时间标签 time index
    # 1.初始随机分配簇 Initialization
    kmeans = KMeans(n_clusters=10,random_state=49,n_init='auto')
    k_data = load_data[:,:int(load_data.shape[1]*train_ratio)]
    kmeans.fit(k_data)
    cluster_list = np.array(kmeans.labels_)
    user_belong = count_clusters(num_cluster=K, cluster_list=cluster_list)
    print(cluster_list)
    for epoch in range(max_epoch):
        original_cluster_list = cluster_list
        aggr_data = aggregate_by_cluster_list(load=load_data,labels=cluster_list,num_cluster=K, agg='mean')
        # 2.MLR预测模型构建 Forecasting Models Training
        model_dict = train_model(cluster_num=K, cluster_list=cluster_list, aggr_data=aggr_data, 
                                lag=48,train_ratio=train_ratio,temp_data=temp_data,time_index=time_index)
        # print(model_dict)
        # 3.构建误差反馈矩阵 Feedback Mechanism
        matrix = error_feedback(load_data=load_data,cluster_num=K, model_dict=model_dict,lag=48,
                            train_ratio=train_ratio, val_ratio=val_ratio, temp_data=temp_data, time_index=time_index)
        # 4.更新簇标签 Updating Clustering Membership
        cluster_list = np.argmin(matrix,axis=1)
        print(cluster_list)
        if change_rate_simple(old_labels=original_cluster_list,new_labels=cluster_list) <= 1:
            print("finish")
            break
    predict_result, test_result = Predict(cluster_num=K,cluster_list=cluster_list,
                                        aggr_data=aggr_data,lag=48,train_ratio=train_ratio,
                                        val_ratio=val_ratio,temp_data=temp_data,
                                        time_index=time_index,model_dict=model_dict)
    unique_elements = np.unique(cluster_list)
    user_belong = count_clusters(num_cluster=K, cluster_list=cluster_list)
    
    for index, cluster_id in enumerate(unique_elements):
        predict_result[index] = predict_result[index]*len(user_belong[str(cluster_id)])
        test_result[index] = test_result[index]*len(user_belong[str(cluster_id)])
    
    np.save(save_path+"predict_result_"+str(data_idx)+".npy",predict_result.sum(axis=0))
    np.save(save_path+"test_result_"+str(data_idx)+".npy",test_result.sum(axis=0))
    
    for index in range(10):
        if model_dict[str(index)] == None:
            pass
        else:
            num +=1
    print("Final Cluster Num = ",num,"\n")

[3 7 5 3 2 5 3 7 5 3 7 9 8 7 5 3 7 5 8 2 5 8 2 5 8 7 5 8 7 5 3 7 5 8 7 5 8
 7 5 8 7 9 8 7 9 0 7 5 3 7 9 8 6 5 8 7 5 3 2 5 8 6 5 8 7 5 3 2 5 8 7 5 8 7
 5 8 6 5 8 7 9 8 2 5 8 2 5 8 7 9 8 6 9 8 7 5 8 6 9 8 7 5 3 7 5 8 7 5 8 2 9
 8 7 5 8 7 5 8 7 5 8 7 5 3 7 5 8 7 5 8 2 5 8 7 5 8 7 9 8 1 9 8 2 9 4 7 9 8
 6 9]
[3 5 5 1 9 5 1 9 9 1 5 5 1 9 5 1 5 5 1 9 5 1 5 5 1 5 9 1 5 9 8 5 5 3 9 5 1
 5 5 1 5 9 1 9 9 1 5 5 3 9 5 8 5 5 1 9 5 3 5 5 1 9 5 1 5 9 1 5 9 1 5 9 1 5
 5 3 5 5 1 5 9 1 9 5 1 5 5 1 9 5 1 9 9 1 5 5 3 5 9 1 5 5 8 9 5 1 5 9 1 5 9
 1 5 5 1 9 9 1 5 9 1 5 5 1 5 9 4 9 5 4 5 5 8 9 9 3 9 5 1 5 5 1 5 9 1 9 5 8
 5 5]
[8 9 9 8 9 9 9 9 9 9 9 9 9 9 9 9 9 5 9 9 9 8 9 9 8 9 9 9 9 9 8 5 9 8 9 9 8
 9 9 9 9 9 8 9 9 8 9 9 8 9 9 8 9 9 9 9 9 8 9 9 8 9 9 8 5 9 8 9 9 8 5 9 8 9
 9 8 9 9 8 9 9 9 9 9 9 5 5 8 9 9 8 9 9 9 9 9 8 9 9 9 9 9 8 9 9 8 5 9 8 9 9
 8 5 9 9 9 9 9 9 9 8 5 9 8 5 9 8 9 5 8 9 9 8 9 9 8 9 9 9 9 9 9 9 9 8 9 9 8
 9 9]
[8 5 5 9 5 5 9 5 5 5 5 5 5 5 5 9 5 5 9 5 5 9 5 5 9 5 5 9 5 5 8 5 5 8 5 5 9
 5 5 9 